In [1]:
from __future__ import print_function

from IPython.display import display, clear_output

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# for data import from files
import pandas as pd

# for open filedialog
from tkinter import filedialog
from tkinter import *

# for plotly graphics (alternative to matplotlib method)
import plotly
from plotly.graph_objs import Scatter, Layout, Data, Surface
import plotly.plotly as py
import plotly.graph_objs as go


import numpy as np
from numpy import *
from numpy import linspace, pi

from scipy.optimize import minimize

In [2]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

def on_button_clicked(b):
    clear_output()
    infile = filedialog.askopenfilename(multiple=False,filetypes = (("text files","*.txt"),("all files","*.*")))
    %gui tk
    print('file name:    ',infile)
    print('first 5 rows of the file:')
    global df1
    df1=pd.read_csv(infile,sep='\t', header=None)
    display(df1.head())
    
# this line is neede only in auto launch whole notebook mode
on_button_clicked(1)
    
out=widgets.Output()
button = widgets.Button(description="Open file")
display(button)
button.on_click(on_button_clicked)

file name:     I:/Cloud/YandexDisk-education@sputnix.ru/Products/OrbiCraft/Tests/Mag_raw_data/mag_K_032_18_filtered.txt
first 5 rows of the file:


,0,1,2
0,104,-686,-298
1,104,-686,-296
2,103,-685,-298
3,104,-687,-297
4,111,-685,-296


In [3]:
def show_3d_point_cloud(df):
    x, y, z = [df[:,i] for i in [0,1,2]]
    trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=4,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.1
            ),
            opacity=0.95
        )
    )
    #just a sphere
    xmean = np.mean(x)
    ymean = np.mean(y)
    zmean = np.mean(z)
    rmean = np.mean(np.sqrt((x-xmean)**2+(y-ymean)**2+(z-zmean)**2))
    print('xmean=',xmean,'   ymean=',ymean,'   zmean=',zmean)
    print('rmean=', rmean)
    theta = linspace(0,2*pi,100)
    phi = linspace(0,pi,100)
    x = rmean*outer(cos(theta),sin(phi))
    y = rmean*outer(sin(theta),sin(phi))
    z = rmean*outer(ones(100),cos(phi))
    spheredata = Data([
        Surface(
            x=x,
            y=y,
            z=z,
            opacity=0.4
        )
    ])
    
    data = [trace1,spheredata[0]]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='simple-3d-scatter.html')
    
dfnp = df1.values
show_3d_point_cloud(dfnp)

xmean= 58.9907284768    ymean= -209.606622517    zmean= -235.315231788
rmean= 447.659201103


In [4]:
# rcal = Acal.rmes+b

def f_mag_cal(params):
    A_cal = params[0:9].reshape(3,3)
    b = params[9:].reshape(3,1)
    
    x, y, z = [df1.iloc[:,i] for i in [0,1,2]]
    xmean = np.mean(x)
    ymean = np.mean(y)
    zmean = np.mean(z)
    rmean = np.mean(np.sqrt((x-xmean)**2+(y-ymean)**2+(z-zmean)**2))
    dfnp = np.array(df1.values)

    df_cal = np.dot(A_cal,dfnp.transpose()).transpose()+b.transpose()
    df_cal_uni = df_cal/rmean
    display(df_cal_uni)
    
    out = np.sum((np.sum(df_cal_uni**2,axis=1)-1)**2)
    print ('out=', out)
    return out
    
f_mag_cal(np.append(np.identity(3),np.zeros((3,1))))


array([[ 0.23231959, -1.53241573, -0.66568497],
       [ 0.23231959, -1.53241573, -0.66121728],
       [ 0.23008574, -1.53018188, -0.66568497],
       ..., 
       [ 0.94268139, -0.58750049, -0.90023839],
       [ 0.97842287, -0.48027607, -0.7840786 ],
       [ 0.98959208, -0.43113154, -0.73270023]])

out= 739.866598715


739.8665987154867

In [5]:
x0 = np.append(np.identity(3),np.zeros((3,1)))
res = minimize(f_mag_cal, x0, tol=1e-3);
clear_output()
print (res.x)

[  6.95209001e-01   5.42125788e-01   6.95154110e-01   4.07311224e-01
   3.46403399e-01  -8.27569844e-01   7.02792939e-01  -6.33178259e-01
  -1.71519949e-01   2.66933723e+02  -1.35725296e+02  -2.05679819e+02]


In [8]:
temp = np.array(res.x)
A_cal = temp[0:9].reshape(3,3)
b = temp[9:].reshape(3,1)
display(A_cal)
display(b)

dfnp = np.array(df1.values)
df_temp = np.dot(A_cal,dfnp.transpose()).transpose()+b.transpose()
show_3d_point_cloud(df_temp)

array([[ 0.695209  ,  0.54212579,  0.69515411],
       [ 0.40731122,  0.3464034 , -0.82756984],
       [ 0.70279294, -0.63317826, -0.17151995]])

array([[ 266.93372294],
       [-135.72529634],
       [-205.67981905]])

xmean= 30.7311023421    ymean= 10.433632565    zmean= 8.85806133707
rmean= 445.740368219
